In [ ]:
library(tidyverse)

In [ ]:
############################################################################
#                                                                          #
#                          SET PARAMETERS HERE                             #
#                                                                          #
############################################################################

############### input parameters (in data folder) ##########################
# input_folder = "all_germline_filtered_bams_tumor_ci5_cs1e9" # "all_germline_ci5_cs1e9" # "ci5_cs1e9" # "cs1e9"
input_folder = "all_germline_filtered_bams_tumor_subtract_other_tumor_ci5_cs1e9"
#filename =  "plotdata_unique_tumor_all_germline_singletons_excluded_tumor_4_cs_filtered.txt" # "plotdata_tumor_cs1e9_filtered.txt" # plotdata_tumor_filtered.txt
filename = "plotdata_unique_tumor_kmers_filtered.txt"

############### output parameters (in plotting_results folder) #############
#output_folder = "ci1_cs1e9" # "ci5_cs1e9" # "union_germline_ci4_cs1e9" # union_germline_filtered_tumor_bams_ci5_cs1e9 # "union_germline_filtered_tumor_bams_subtract_other_tumors_ci5_cs1e9"
output_folder = "union_germline_filtered_tumor_bams_subtract_other_tumors_ci5_cs1e9"
#plot_title = "Unique_tumor_kmers" # "Tumor_kmers"
plot_title = "Unique_tumor_kmers"


paramspace <- read.csv("../data/metadata/paramspace_phaseI.csv", sep = ",")
dim(paramspace)
#output_path_unique_kmers, pt_id

In [ ]:
data_all <- NULL

for (row in 1:nrow(paramspace)){
    pt = paramspace[row, "pt_id"]
    print(pt)
    fd = paramspace[row, "output_path_unique_kmers"]
    
    data <- read.table(paste("../data/", pt, "/", fd ,"/", input_folder, "/", filename, sep = ""))
    data <- data %>% mutate(patient_id = pt)
    
    cum_sum <- NULL
    for (row in 1:nrow(data)){
        if (row <= 1000){
            sub_data <- data[row:nrow(data), ]
            cum_sum <- c(cum_sum, sum(sub_data$V2))
        } else {
            cum_sum <- c(cum_sum, NA)
        }
        }

    data <- data %>% mutate(cumulative_kmers = cum_sum)
    
    data_all <- rbind(data_all, data)
    }



In [ ]:
dim(data_all)

colnames(data_all) <- c("count", "n", "patient_id", "cumulative_kmers")
head(data_all)

In [ ]:
data_all_1000 <- data_all %>% filter(count <= 250)
data_all_1000$label <- NA
data_all_1000$label[which(data_all_1000$count == max(data_all_1000$count))] <- data_all_1000$patient_id[which(data_all_1000$count == max(data_all_1000$count))]


In [ ]:
#data_only_500[1, "patient_id"]
data_only_250 <- data_all_1000 %>% filter(count == 125)
data_only_250
log(max(data_only_250$n))

In [ ]:


line_smooth <- ggplot(data_all_1000) + 
    #geom_smooth(aes(count, log10(n), color = patient_id), size = 0.5, alpha = 1, se = FALSE) + 
    geom_line(aes(count, log10(n), color = patient_id), size = 0.4, alpha = 0.7) + 
    xlab("Number of times k-mer is observed") + 
    ylab("log(Number of k-mers)") + 
    theme_minimal()  + 
    annotate("text", x = 150, y = log10(max(data_only_250$n)) + 0.6, label = data_only_250$patient_id[which(data_only_250$n == max(data_only_250$n))]) +
    ggtitle(paste(plot_title, ", pipeline ", input_folder, sep =  ""))

line_smooth



In [ ]:
line_cumulative <- ggplot(data_all_1000) + 
    #geom_smooth(aes(count, log10(cumulative_kmers), color = patient_id), size = 0.5, alpha = 1, se = FALSE) + 
    geom_line(aes(count, log10(cumulative_kmers), color = patient_id), size = 0.5, alpha = 1) + 
    xlab("Number of times k-mer is observed") + 
    ylab("log(Cumulative number of kmers (excluding everything < x)") + 
    theme_minimal()  + 
    annotate("text", x = 125, y = log10(max(data_only_250$cumulative_kmers)) + 0.6, label = data_only_250$patient_id[which(data_only_250$n == max(data_only_250$n))]) +
    ggtitle(paste(plot_title, ", pipeline ", input_folder, sep =  ""))

line_cumulative

In [ ]:
ggsave(
  paste("../plotting_results/", output_folder, "/kmer_distributions/", plot_title, "_count_distribution.png", sep = ""),
  line_smooth, dpi = "print")

ggsave(
  paste("../plotting_results/", output_folder, "/kmer_distributions/", plot_title, "_count_distribution_cumulative.png", sep = ""),
  line_cumulative, dpi = "print")